<a href="https://colab.research.google.com/github/intellygentle/Alignedlayer-Testnet-Proof-/blob/main/SadTalker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# 1. Reset to a stable directory
%cd /content

# 2. Install Python 3.8 and the VENV module (required to get pip)
!apt-get update -y
!apt-get install -y software-properties-common
!add-apt-repository -y ppa:deadsnakes/ppa
!apt-get update -y
!apt-get install -y python3.8 python3.8-dev python3.8-distutils python3.8-venv

# 3. Bootstrap pip for Python 3.8
# Using ensurepip is the safest way to get pip into a specific python binary
!python3.8 -m ensurepip
!python3.8 -m pip install --upgrade pip setuptools wheel

# 4. Set alternatives so "python" and "python3" point to 3.8
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!update-alternatives --install /usr/local/bin/python python /usr/bin/python3.8 2

# 5. Clone and Setup SadTalker
!rm -rf /content/SadTalker
!git clone https://github.com/cedro3/SadTalker.git /content/SadTalker
%cd /content/SadTalker
os.environ['PYTHONPATH'] = '/content/SadTalker:' + os.environ.get('PYTHONPATH', '')

# 6. Install Torch (Must use python3.8 explicitly here)
!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113

# 7. Install remaining requirements
!apt-get install -y ffmpeg
!python3.8 -m pip install -r requirements.txt

print("\nSuccessfully installed Python 3.8 and pip!")
!python --version


/content
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,302 kB]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [354 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:13 http://security.ubun

In [2]:
# Install gTTS specifically for your Python 3.8 environment
!python3.8 -m pip install gTTS

In [9]:
!python -m pip install gTTS


  Using cached gTTS-2.5.4-py3-none-any.whl.metadata (4.1 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
Using cached gTTS-2.5.4-py3-none-any.whl (29 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1


In [15]:
# Install edge-tts for high-quality male/female voices
!python -m pip install edge-tts

  Using cached edge_tts-7.2.7-py3-none-any.whl.metadata (5.5 kB)
Using cached edge_tts-7.2.7-py3-none-any.whl (30 kB)


In [21]:
import os
import edge_tts
import asyncio
from IPython.display import Audio, display

# If not installed yet, uncomment the next line:
# !pip install nest_asyncio

import nest_asyncio
nest_asyncio.apply()

#@title **Text to Audio (TTS)**
text_input = "Assalamu alaikum! Dive into another captivating episode of \"WISDOM OF A BELIEVER,\" where we unravel the pulse of modern Islamic issues and arm you with timeless truths to shield your faith against the tidal waves of today's deceptions—like neo-atheism, evolutionary myths, secular tides, and all the clever illusions crafted to pull souls from the path of devotion. Join the journey—stick around, and thank you!" #@param {type:"string"}
gender = "Male" #@param ["Male", "Female"]
output_audio_filename = "generated_audio.wav" #@param {type:"string"}

voice_map = {
    "Male": "en-US-GuyNeural",
    "Female": "en-US-JennyNeural"
}

driven_audio_dir = '/content/SadTalker/examples/driven_audio'
os.makedirs(driven_audio_dir, exist_ok=True)
output_path = os.path.join(driven_audio_dir, output_audio_filename)

async def generate_speech():
    communicate = edge_tts.Communicate(text_input, voice_map[gender])
    await communicate.save(output_path)

try:
    print(f"Generating {gender} voice...")
    loop = asyncio.get_event_loop()
    loop.run_until_complete(generate_speech())

    print(f"✅ Audio saved successfully to: {output_path}")
    display(Audio(output_path))

    audio = output_audio_filename
    print(f"The 'audio' variable is set to: {audio}")

except Exception as e:
    print(f"❌ An error occurred: {e}")

Generating Male voice...
✅ Audio saved successfully to: /content/SadTalker/examples/driven_audio/generated_audio.wav


The 'audio' variable is set to: generated_audio.wav


In [ ]:
!rm -rf checkpoints
!bash scripts/download_models.sh

In [28]:
#@title **inference for face**
image ='image6.jpg' #@param {type:"string"}
audio ='generated_audio.wav' #@param {type:"string"}
source_image = 'examples/source_image/' + image
driven_audio = 'examples/driven_audio/' + audio

!python3.8 inference.py --driven_audio $driven_audio \
           --source_image $source_image \
           --result_dir ./results --enhancer gfpgan

using safetensor as default
3DMM Extraction for source image
landmark Det:: 100% 1/1 [00:00<00:00, 16.42it/s]
3DMM Extraction In Video:: 100% 1/1 [00:00<00:00, 24.24it/s]
mel:: 100% 582/582 [00:00<00:00, 42264.75it/s]
audio2exp:: 100% 59/59 [00:00<00:00, 338.45it/s]
Face Renderer:: 100% 291/291 [02:41<00:00,  1.80it/s]
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (256, 254) to (256, 256) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
The generated video is named ./results/2026_01_21_16.40.43/image6##generated_audio.mp4
face enhancer....
Face Enhancer:: 100% 582/582 [03:06<00:00,  3.13it/s]
The generated video is named ./results/2026_01_21_16.40.43/image6##generated_audio_enhanced.mp4
The generated video is named: ./results/2026_01_21_16.40.43.mp4


In [29]:
#@title **play movie**
import glob
from IPython.display import HTML
from base64 import b64encode
import os, sys

# get the last from results
mp4_name = sorted(glob.glob('./results/*.mp4'))[-1]

mp4 = open('{}'.format(mp4_name),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

print('Display animation: {}'.format(mp4_name), file=sys.stderr)
display(HTML("""
  <video width=256 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))

Display animation: ./results/2026_01_21_16.40.43.mp4


In [32]:
#@title **🎬 SadTalker Video Rendering (Full Control)**

#@markdown ### 📁 **Input Files**
image = 'image6.jpg' #@param {type:"string"}
audio = 'generated_audio.wav' #@param {type:"string"}

#@markdown ---
#@markdown ### 🎭 **Animation Style**
pose_style = 5 #@param {type:"slider", min:0, max:45, step:1}
#@markdown - **0** = Neutral/natural (recommended for portraits)
#@markdown - **1-45** = Different preset head movements & expressions

expression_scale = 1.0 #@param {type:"slider", min:0.0, max:3.0, step:0.1}
#@markdown - **1.0** = Normal intensity
#@markdown - **1.5-2.0** = More expressive/dramatic
#@markdown - **0.5-0.8** = Subtle/calm

still_mode = True #@param {type:"boolean"}
#@markdown ✅ **Enable for photos** (reduces unnatural head bobbing)

#@markdown ---
#@markdown ### 🖼️ **Image Processing**
preprocess = "full" #@param ["crop", "resize", "full", "extcrop", "extfull"]
#@markdown - **crop** = Face only (tight crop, removes background)
#@markdown - **full** = Entire image preserved (best for portraits)
#@markdown - **extfull** = Extended tracking (for full-body shots)

#@markdown ---
#@markdown ### ✨ **Quality Enhancement**
use_enhancer = True #@param {type:"boolean"}
enhancer = "gfpgan" #@param ["gfpgan", "RestoreFormer"]
#@markdown - **gfpgan** = Sharper face details (recommended)
#@markdown - **RestoreFormer** = Alternative enhancer

#@markdown ---
#@markdown ### 🎥 **Advanced Settings**
batch_size = 2 #@param {type:"slider", min:1, max:8, step:1}
#@markdown Higher = Faster rendering (needs more VRAM)

size = 512 #@param [256, 512] {type:"integer"}
#@markdown Output resolution (512 = higher quality but slower)

fps = 20 #@param {type:"slider", min:20, max:30, step:5}
#@markdown Frames per second (25 = smooth, standard)

#@markdown ---
#@markdown ### 🔧 **Experimental Features**
use_idle_mode = False #@param {type:"boolean"}
#@markdown Adds blinking/micro-movements when not speaking

use_ref_video = False #@param {type:"boolean"}
ref_video_path = "" #@param {type:"string"}
#@markdown Copy head movements from a reference video

#@markdown ---

# Build paths
source_image = f'examples/source_image/{image}'
driven_audio = f'examples/driven_audio/{audio}'

# Construct the command
cmd = f"""python3.8 inference.py \
    --driven_audio {driven_audio} \
    --source_image {source_image} \
    --result_dir ./results \
    --pose_style {pose_style} \
    --expression_scale {expression_scale} \
    --preprocess {preprocess} \
    --size {size} \
    --batch_size {batch_size} \
    --fps {fps}"""

# Add optional flags
if still_mode:
    cmd += " --still"

if use_enhancer:
    cmd += f" --enhancer {enhancer}"

if use_idle_mode:
    cmd += " --use_idle_mode"

if use_ref_video and ref_video_path:
    cmd += f" --ref_eyeblink {ref_video_path} --ref_pose {ref_video_path}"

# Display settings summary
print("="*60)
print("🎬 RENDERING SETTINGS")
print("="*60)
print(f"📸 Source Image: {image}")
print(f"🎵 Audio: {audio}")
print(f"🎭 Pose Style: {pose_style} | Expression: {expression_scale}x")
print(f"🖼️  Preprocess: {preprocess} | Still Mode: {still_mode}")
print(f"✨ Enhancer: {enhancer if use_enhancer else 'Disabled'}")
print(f"🎥 Resolution: {size}px | FPS: {fps} | Batch: {batch_size}")
print("="*60)
print("\n🚀 Starting render...\n")

# Run the command
!{cmd}

print("\n✅ Rendering complete! Check ./results folder")

🎬 RENDERING SETTINGS
📸 Source Image: image6.jpg
🎵 Audio: generated_audio.wav
🎭 Pose Style: 5 | Expression: 1.0x
🖼️  Preprocess: full | Still Mode: True
✨ Enhancer: gfpgan
🎥 Resolution: 512px | FPS: 25 | Batch: 2

🚀 Starting render...

usage: inference.py
       [-h]
       [--driven_audio DRIVEN_AUDIO]
       [--source_image SOURCE_IMAGE]
       [--ref_eyeblink REF_EYEBLINK]
       [--ref_pose REF_POSE]
       [--checkpoint_dir CHECKPOINT_DIR]
       [--result_dir RESULT_DIR]
       [--pose_style POSE_STYLE]
       [--batch_size BATCH_SIZE]
       [--size SIZE]
       [--expression_scale EXPRESSION_SCALE]
       [--input_yaw INPUT_YAW [INPUT_YAW ...]]
       [--input_pitch INPUT_PITCH [INPUT_PITCH ...]]
       [--input_roll INPUT_ROLL [INPUT_ROLL ...]]
       [--enhancer ENHANCER]
       [--background_enhancer BACKGROUND_ENHANCER]
       [--cpu]
       [--face3dvis]
       [--still]
       [--preprocess {crop,extcrop,resize,full,extfull}]
       [--verbose]
       [--old_version]
     

In [ ]:
#@title **inference for portrait**
image ='full3.png' #@param {type:"string"}
audio ='eluosi.wav' #@param {type:"string"}
source_image = 'examples/source_image/' + image
driven_audio = 'examples/driven_audio/' + audio

!python3.8 inference.py --driven_audio $driven_audio \
           --source_image $source_image \
           --result_dir ./results --still --preprocess full --enhancer gfpgan

In [ ]:
#@title **play movie**
import glob
from IPython.display import HTML
from base64 import b64encode
import os, sys

# get the last from results
mp4_name = sorted(glob.glob('./results/*.mp4'))[-1]

mp4 = open('{}'.format(mp4_name),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

print('Display animation: {}'.format(mp4_name), file=sys.stderr)
display(HTML("""
  <video width=256 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))